In [1]:
#importing libraries and modules
import time
import datetime
import hashlib
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
from time import gmtime, strftime
import os
import csv
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [2]:
#function for genrating the list that contains the directory folders names
def item_list():
    #request and open the required url
    url = urlopen("http://125.19.52.218/realtime/")

    #make BeautifulSoup object
    soup = BeautifulSoup(url, "html.parser")

    #find the needed info using bs4 object
    m3 = soup.find_all("div", {"class" : "tableData"})

    master_list1 = []

    #create list of water sample other details
    for element in m3:
        temp_dic={}
        m4=element.find_all("tr")
        for ele in m4:
            try:
                master_list1.append(ele.find("td").text.strip())
            except:
                continue
    list_set = set(master_list1)
    master_list = list(list_set)
    
    #making a csv file of parameter names for future use
    master_list2 = []
    for para in master_list:
        temp_dic2 = {}
        temp_dic2['Parameters'] = para
        master_list2.append(temp_dic2)
    d = pd.DataFrame(master_list2)
    d.to_csv("Downloads/Runtime/Parameter's.csv",index=True)
    
    names = []
    for i in master_list:
        i = i.replace("/","_")
        names.append(i)
    return names
#calling the item_list function to create the parameter csv file
#item_list()

In [3]:
#function for creating the required directories at specified location
def create_dir():
    master_list = item_list()
    
    #create the folder where u want to start creation of files
    directory = "Runtime"
    parent_dir = "Downloads/"
    path = os.path.join(parent_dir, directory)    
    #create a main directory
    os.mkdir(path)
    
    for i in master_list:
        try:
            directory = i
            #passing the loction where u want to create the files
            parent_dir ="Downloads/Runtime/"
            path = os.path.join(parent_dir, directory)
            #creating a main directory folder
            os.mkdir(path)
        except:
            continue

        for j in range(1,35):
            directory1 = "Location" + str(j)
            #passing the location where the we want to create the sub-files
            parent_dir = "Downloads/Runtime/" + i
            path = os.path.join(parent_dir, directory1)
            #creating sub-directories of a mail directory folder
            os.mkdir(path)
#calling the create_dir function
#create_dir()

In [4]:
def create_graph():
    #calling the item-list function and replacing the list names element with specified value
    master_list3 = item_list()
    names=[]
    for j in master_list3:
        j = j.replace("_","/")
        names.append(j)
    l = 0
    df5=pd.read_csv("Downloads/Runtime/Location's.csv")
    d3=df5["Locations"].tolist()
    for i in master_list3:
        for loc in range(1,(len(d3)+1)):
            try:
                plt.figure(figsize=(14,11))
                plt.title('Analysis of Parameter: {}'.format(i), fontdict={ 'fontsize': 23 })
                plt.ylabel(names[l], fontdict={ 'fontsize': 18 })
                plt.xlabel("Time", fontdict={ 'fontsize': 18 })
                df = pd.read_csv("Downloads/Runtime/{}/Location{}/Data.csv".format(i,str(loc)))
                g = sns.lineplot(x = "DateTime", y = names[l],color="red", data = df)
                g = sns.scatterplot(x = "DateTime", y = names[l],marker="v",color="red", data = df, s=100)
                labels = g.get_xticklabels()
                plt.setp(labels, rotation = 90, horizontalalignment = 'right')
                plt.savefig('Downloads/Runtime/{}/Location{}/Data.jpeg'.format(i,str(loc)))
            except Exception as e:
                print("error : ", e)
        l = l + 1
#create_graph()

In [5]:
def create_graph2(name,loc):
    name2 = name.replace("_","/")
    try:
        plt.figure(figsize=(14,11))
        plt.title('Analysis of Parameter: {}'.format(name2), fontdict={ 'fontsize': 23 })
        plt.ylabel(name2, fontdict={ 'fontsize': 18 })
        plt.xlabel("Time", fontdict={ 'fontsize': 18 })
        df = pd.read_csv("Downloads/Runtime/{}/Location{}/Data.csv".format(name,str(loc)))
        g = sns.lineplot(x = "DateTime", y = name2,color="red", data = df)
        g = sns.scatterplot(x = "DateTime", y = name2,marker="v",color="red", data = df, s=100)
        labels = g.get_xticklabels()
        plt.setp(labels, rotation = 90, horizontalalignment = 'right')
        plt.savefig('Downloads/Runtime/{}/Location{}/Data.jpeg'.format(name,str(loc)),bbox_inches='tight')
    except Exception as e:
        print("error : ", e)
#create_graph2("Potassium(mg_l)",1)

In [6]:
#function for generating the location and data csv files for the first time use 
def create_csv():
    #request and open the required url
    url = urlopen("http://125.19.52.218/realtime/")
    #make BeautifulSoup object
    soup = BeautifulSoup(url, "html.parser")
    
    #find the needed info using bs4 object
    m2 = soup.find_all("h3")
    m3 = soup.find_all("div", {"class" : "tableData"})
    
    #create datetime object
    now = datetime.datetime.now()
    
    #initilizing the variable k
    k = 1
    
    #calling the item-list function and replacing the list names element with specified value
    master_list3 = item_list()
    names=[]
    for j in master_list3:
        j = j.replace("_","/")
        names.append(j)
        
    #create list of water sample other details
    for element in m3:
        temp_dic={}
        l = 0
        m4=element.find_all("tr")
        for ele in m4:
            try:
                #finding the required data
                temp_dic[ele.find("td").text.strip()]=ele.find_all("td")[-1].text.strip()
            except:
                continue
        #checking whether the dictionary is empty or not
        if temp_dic!={}:
            for i in names:
                l1 = []
                temp_dic2 = {}
                temp_dic2["DateTime"] = now.strftime('%d %b %H:%M')
                temp_dic2["Date"] = now.strftime('%d %b %Y')
                temp_dic2["Time"] = now.strftime('%H:%M:%S')
                temp_dic2[i]=temp_dic.get(i,0.0)
                #print(temp_dic2[i])
                
                #adding dictionary to a list
                l1.append(temp_dic2)
                
                #creating a dataframe
                df = pd.DataFrame(l1)
                #display(df)
                #generating the csv files of the required data
                df.to_csv('Downloads/Runtime/{}/Location{}/Data.csv'.format(master_list3[l],str(k)),index=True)
                l = l + 1
            k = k + 1
#create_csv()

In [7]:
#function for generating the updated csv files for the scraped data 
def create_csv2():
    #request and open the required url
    url = urlopen("http://125.19.52.218/realtime/")
    #make BeautifulSoup object
    soup = BeautifulSoup(url, "html.parser")
    
    #find the needed info using bs4 object
    m3 = soup.find_all("div", {"class" : "tableData"})
    
    #create datetime object
    now = datetime.datetime.now()
    
    #initilizing the variable k
    k = 1
    
    #calling the item-list function and replacing the list names element with specified value
    master_list3 = item_list()
    names=[]
    for j in master_list3:
        j = j.replace("_","/")
        names.append(j)
        
    #create list of water sample other details
    for element in m3:
        temp_dic={}
        l = 0
        m4=element.find_all("tr")
        for ele in m4:
            try:
                #finding the required data
                temp_dic[ele.find("td").text.strip()]=ele.find_all("td")[-1].text.strip()
            except:
                continue
        #checking whether the dictionary is empty or not
        if temp_dic!={}:
            for i in names:
                l1 = []
                temp_dic2 = {}
                temp_dic2["DateTime"] = now.strftime('%d %b %H:%M')
                temp_dic2["Date"] = now.strftime('%d %b %Y')
                temp_dic2["Time"] = now.strftime('%H:%M:%S')
                temp_dic2[i]=temp_dic.get(i,"")
                
                #read the previously generated csv file
                df1=pd.read_csv('Downloads/Runtime/{}/Location{}/Data.csv'.format(master_list3[l],str(k)))
                #converting the data into list of dictionary
                a=df1.to_dict("records")
                try:
                    #eleminating unwanted column generated due to the "".read_csv method"
                    del a[-1]["Unnamed: 0"]
                except:
                    pass
                try:
                    x = float(temp_dic2[i])
                except:
                    #chaning the nan/empty string with a value(0.0)
                    temp_dic2[i]=0.0
                
                #checking for the updation of data    
                if float(a[-1].get(i,"0.0")) != float(temp_dic2.get(i,"0.0")): 
                    #adding updated data to list of dictionary
                    a.append(temp_dic2)
                    #craete a panda dataframe
                    df2=pd.DataFrame(a)
                    #showing the changes that are updated in a csv file
                    display(df2)
                    #genrating the updated data csv file
                    df2.to_csv('Downloads/Runtime/{}/Location{}/Data.csv'.format(master_list3[l],str(k)), index=False)
                    create_graph2(master_list3[l],k)
                    time.sleep(5)
                l = l + 1
            k = k + 1

In [8]:
#function for checking whether data is updated on a site or not
def notifyer():
    # setting the URL we want to monitor
    url = Request('http://125.19.52.218/realtime/', headers={'User-Agent': 'Mozilla/5.0'})

    print("running")

    #monitering the website
    while True:
        try:
            #open file in read mode and get the old hashcode of the site
            file = open("hash2code.txt","r")
            currentHash = file.readline()
            file.close()

            # perform the get request
            response = urlopen(url).read()

            # create a new hash
            newHash = hashlib.sha224(response).hexdigest()

            # check if new hash is same as the previous hash
            if newHash == currentHash:
                print("Data not updated")
                print("process done.....")
                print("next process starts.....")
                
                # wait for 10 seconds
                time.sleep(3600)
                continue

            # if something changed in the hashes
            else:
                # notify
                print("Data updated")
                create_csv2()
                time.sleep(20)

                #create a file, write it and close it
                file = open("hash2code.txt","w+")
                file.write(newHash)
                file.close()
                
                print("process done.....")
                print("next process starts.....")
                
                # wait for 10 seconds
                time.sleep(3600)
                continue

        #handling exceptions
        except Exception as e:
            print("error",e)
            continue

In [ ]:
#defining the main function
if __name__ == '__main__':
    #calling the notifyer function to check the status of the scraped data
    notifyer()

running
Data updated


,Unnamed: 0,DateTime,Date,Time,pH
0,0.0,04 Aug 14:26,04 Aug 2021,14:26:24,7.63
1,1.0,09 Aug 11:41,09 Aug 2021,11:41:48,7.75
2,2.0,09 Aug 12:57,09 Aug 2021,12:57:04,7.76
3,3.0,09 Aug 13:57,09 Aug 2021,13:57:19,7.77
4,4.0,09 Aug 15:16,09 Aug 2021,15:16:19,7.75
5,5.0,09 Aug 16:29,09 Aug 2021,16:29:24,7.76
6,6.0,09 Aug 17:29,09 Aug 2021,17:29:51,7.77
7,7.0,09 Aug 18:30,09 Aug 2021,18:30:16,7.75
8,8.0,09 Aug 19:30,09 Aug 2021,19:30:40,7.76
9,9.0,10 Aug 00:49,10 Aug 2021,00:49:39,7.75


,Unnamed: 0,DateTime,Date,Time,Conductivity(µS/cm)
0,0.0,04 Aug 14:26,04 Aug 2021,14:26:24,291
1,1.0,09 Aug 11:41,09 Aug 2021,11:41:48,159
2,2.0,09 Aug 12:57,09 Aug 2021,12:57:04,158
3,3.0,09 Aug 13:57,09 Aug 2021,13:57:19,159
4,4.0,09 Aug 15:16,09 Aug 2021,15:16:19,160
5,5.0,09 Aug 16:29,09 Aug 2021,16:29:24,159
6,6.0,09 Aug 17:29,09 Aug 2021,17:29:51,160
7,7.0,10 Aug 00:49,10 Aug 2021,00:49:39,161
8,8.0,10 Aug 08:24,10 Aug 2021,08:24:01,168
9,9.0,10 Aug 10:27,10 Aug 2021,10:27:28,169


,Unnamed: 0,DateTime,Date,Time,Nitrate(mg/l)
0,0.0,04 Aug 14:26,04 Aug 2021,14:26:24,0.52
1,1.0,09 Aug 11:41,09 Aug 2021,11:41:48,0.59
2,2.0,09 Aug 12:57,09 Aug 2021,12:57:04,0.16
3,3.0,09 Aug 13:57,09 Aug 2021,13:57:19,0.82
4,4.0,09 Aug 15:16,09 Aug 2021,15:16:19,0.71
5,5.0,09 Aug 16:29,09 Aug 2021,16:29:24,0.87
6,6.0,09 Aug 17:29,09 Aug 2021,17:29:51,0.67
7,7.0,09 Aug 18:30,09 Aug 2021,18:30:16,0.66
8,8.0,09 Aug 19:30,09 Aug 2021,19:30:40,0.8
9,9.0,10 Aug 00:49,10 Aug 2021,00:49:39,0


,Unnamed: 0,DateTime,Date,Time,BOD(mg/l)
0,0.0,04 Aug 14:26,04 Aug 2021,14:26:24,2.24
1,1.0,09 Aug 11:41,09 Aug 2021,11:41:48,5.39
2,2.0,09 Aug 12:57,09 Aug 2021,12:57:04,6.09
3,3.0,09 Aug 13:57,09 Aug 2021,13:57:19,5.29
4,4.0,09 Aug 15:16,09 Aug 2021,15:16:19,4.94
5,5.0,09 Aug 16:29,09 Aug 2021,16:29:24,5.4
6,6.0,09 Aug 17:29,09 Aug 2021,17:29:51,5.3
7,7.0,09 Aug 18:30,09 Aug 2021,18:30:16,5.68
8,8.0,09 Aug 19:30,09 Aug 2021,19:30:40,5.45
9,9.0,10 Aug 00:49,10 Aug 2021,00:49:39,0


...

,Unnamed: 0,DateTime,Date,Time,DO(mg/l)
0,0.0,04 Aug 14:26,04 Aug 2021,14:26:24,1.73
1,1.0,09 Aug 11:41,09 Aug 2021,11:41:48,2.99
2,2.0,09 Aug 12:57,09 Aug 2021,12:57:04,3.52
3,3.0,09 Aug 13:57,09 Aug 2021,13:57:19,2.77
4,4.0,09 Aug 15:16,09 Aug 2021,15:16:19,3.65
5,5.0,09 Aug 16:29,09 Aug 2021,16:29:24,3.54
6,6.0,09 Aug 17:29,09 Aug 2021,17:29:51,3.57
7,7.0,09 Aug 18:30,09 Aug 2021,18:30:16,3.23
8,8.0,09 Aug 19:30,09 Aug 2021,19:30:40,3.28
9,9.0,10 Aug 00:49,10 Aug 2021,00:49:39,2.84


,Unnamed: 0,DateTime,Date,Time,COD(mg/l)
0,0.0,04 Aug 14:26,04 Aug 2021,14:26:24,21.1
1,1.0,09 Aug 11:41,09 Aug 2021,11:41:48,21.11
2,2.0,09 Aug 12:57,09 Aug 2021,12:57:04,20.72
3,3.0,09 Aug 13:57,09 Aug 2021,13:57:19,20.23
4,4.0,09 Aug 15:16,09 Aug 2021,15:16:19,19.94
5,5.0,09 Aug 16:29,09 Aug 2021,16:29:24,20.17
6,6.0,09 Aug 17:29,09 Aug 2021,17:29:51,20.84
7,7.0,09 Aug 18:30,09 Aug 2021,18:30:16,20.92
8,8.0,09 Aug 19:30,09 Aug 2021,19:30:40,20.91
9,9.0,10 Aug 00:49,10 Aug 2021,00:49:39,19.78


,Unnamed: 0,DateTime,Date,Time,Temperature(°C)
0,0.0,04 Aug 14:26,04 Aug 2021,14:26:24,30.5
1,1.0,09 Aug 11:41,09 Aug 2021,11:41:48,31
2,2.0,09 Aug 13:57,09 Aug 2021,13:57:19,31.1
3,3.0,09 Aug 15:16,09 Aug 2021,15:16:19,31.2
4,4.0,09 Aug 16:29,09 Aug 2021,16:29:24,31.1
5,5.0,09 Aug 17:29,09 Aug 2021,17:29:51,31.2
6,6.0,09 Aug 19:30,09 Aug 2021,19:30:40,31.1
7,7.0,10 Aug 08:24,10 Aug 2021,08:24:01,31
8,8.0,10 Aug 09:26,10 Aug 2021,09:26:56,31.1
9,9.0,10 Aug 10:27,10 Aug 2021,10:27:28,31.4


,Unnamed: 0,DateTime,Date,Time,TSS(mg/l)
0,0.0,04 Aug 14:26,04 Aug 2021,14:26:24,143.29
1,1.0,09 Aug 11:41,09 Aug 2021,11:41:48,146.15
2,2.0,09 Aug 12:57,09 Aug 2021,12:57:04,144.89
3,3.0,09 Aug 13:57,09 Aug 2021,13:57:19,143.04
4,4.0,09 Aug 15:16,09 Aug 2021,15:16:19,141.79
5,5.0,09 Aug 16:29,09 Aug 2021,16:29:24,143.02
6,6.0,09 Aug 17:29,09 Aug 2021,17:29:51,145.92
7,7.0,09 Aug 18:30,09 Aug 2021,18:30:16,146.85
8,8.0,09 Aug 19:30,09 Aug 2021,19:30:40,147.58
9,9.0,10 Aug 00:49,10 Aug 2021,00:49:39,142.27


process done.....
next process starts.....
